<a href="https://colab.research.google.com/github/joshuwaifo/A-Bible-Pre-trained-Transformer-Model/blob/main/QLoraExample_Walkthrough_WithCode_BibleGPT_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In training: learn how to update the pre-trained weight matrix


LoRA: factorise weight update matrix into 2 smaller matrices

There will be multiple layers of these in the large language model

Example if weight matrix was 10 by 10 = 100 parameters

Might be factorised into 10 by 2 and 2 by 10 matrix => 20 + 20 parameters = 40 parameters

This approach can help save a lot of the parameters that are needed

This also corresponds to the time saved too:

Full Finetuning can go from 8 hours plus on multiple GPUs to 1 hour plus on a single GPU with LoRA


And memory too:

Full finetuning 39+ GB per GPU of memory to 16+ GB on a single GPU with LoRA and almost to 12+ GB with qLoRA



In [ ]:
# CPU offloading transferring parametrs from GPU to CPU

QLoRA: stands for Quantized LoRA, Quantized Low-Rank Approximation

In [ ]:
# To do it in practice here are some ways

# Lit-GPT: fully open sourced repository from Lightning AI
# Very hackable as code/source code is present

Setup process, probably need to run this elsewhere ie Replit or on my local machine


In [3]:
# git clone https://github.com/Lightning-AI/lit-gpt
# cd lit-gpt
# pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


Inside cd lit-gpt contains:

- LICENSE
- checkpoints
- finetune
- notebooks
- quantize
- setup.py
- xla
- README.md
- data
- generate
- out
- requirements.txt
- tests
- chat
- eval
- lit_gpt
- pretrain
- scripts
- tutorials

In [ ]:
# tutorials recommended as a way to go from Sebastian

# helps with dealing with out of memory issues etc.

After cloning the litGPT repository and installing the requirements:

- next: choose a model (ie Falcon, Llama, Mistral, StableLM etc.)

- download the model:

In [ ]:
# Example of model download (Mistral 7 Billion parameter model)
# python scripts/download.py --repo_id mistralai/Mistral-7B-Instruct-v0.1

Convert the downloaded weights into an appropriate format

In [ ]:
# Example
# python scripts/convert_hf_checkpoint.py --checkpoint_dir checkpoints/mistralai/Mistral-7B-Instruct-v0.1

Prepare a dataset: this step is necessary with a specific model as each model may have a different tokeniser so make sure things match up

The dataset is therefore then prepared using a specific model tokenizer

In [ ]:
# A lot of examples exist on the Lit-GPT pagehere are pretraining and finetuning examples too
# And there are tutorials for preparing your own custom dataset

# from a CSV file or alpaca type dataset etc.
# CSV file: instructions and outputs format

# example using the alpaca dataset

# python scripts/prepare_alpaca.py --checkpoint_dir checkpoints/mistralai/Mistral-7B-Instruct-v0.1


Finetuning step

Example below is by using

- LoRA
- 16 bit brain float precision (this helps a lot with memory constraints and computation time in comparison to using 32 bits without sacrificing too much in performance)
- Quantization also used here using normal float 4 (to save memory on the order of gigabytes, but sometimes takes a few percentages more computation time)

This is effectively QLoRA

Then depending on GPU the model would train for a while, seeing things like milliseconds per iteration

Total time could be on the order of an hour to a few hours

Which isn't bad for doing 50000 iterations

In [4]:
# Example for doing QLoRA finetuning
# python finetune/lora.py --checkpoint_dir checkpoints/mistralai/Mistral-7B-Instruct-v0.1 --precision bf16-true --quantize "bnb.nf4"

For more efficient techniques of finetuning ie

- finding what a good dataset to train on is

- useful tweaks to LoRA

Check out: NeurIPS Large Language Model Efficiency Challenge:

- 1 LLM + 1 GPU + 1 Day

In [ ]:
# Lit-GPT is the official starter kit to emphasise validity

# https://llm-efficiency-challenge.github.io/

# https://github.com/Lightning-AI/lit-gpt/blob/main/tutorials/neurips_challenge_quickstart.md



Learning more about the instructor that motivated this:

- @rasbt
- sebastian@lightning.ai
- https://sebastianraschka.com
- https://lightning.ai

In [5]:
# Lightning AI: Creators of PyTorch Lightning

Pretraining and Finetuning pipeline

Large unlabelled general text corpus

Pretrained LLM also known as Foundation model (this is normally as practitioners downloaded from a model hub or something similar)

If we have a specific problem or domain ie Medical or Finance

- we might train the LLM further on a domain specific data



In [ ]:
# There are many ways of instruction finetuning

# For example:
# supervised instruction finetuning
# Reinforcement Learning with Human Feedback
# Direct Preference Optimisation

In supervised instruction finetuning

we have a dataset consisting of instruction - input (optional) - output

To make this more efficient:

Use Paraemter Efficient Finetuning techniques like (Q)LoRA




In [ ]:
# Imagine the regular finetuning setup where

# You want to update the pretrained weights

# Imaging weight update was saved separately

# Then you can think of LoRA as a Low-rank approximation or decomposition of that separately saved weight matrix

# So instead of learning a full weight update matrix

# LoRA is learning the decomposed matrices involved in the replacement process

# The trick being we have this rank and we can choose a very small rank for these matrices

# Which helps one to save a lot of parameters

# Making the training more efficient



Finding: with low rank decomposition we can approximate the results of the full weight update

In [ ]:
# In practice there are a lot of settings/tweaks/hyperparameters we can change

# For example:

# what layers you want to apply LoRA
# scaling factor alpha


lora_r = 8
lora_alpha = 16
lora_dropout = 0.05

# query, key and value of an attention head
lora_query = True
lora_key = False
lora_value = True
lora_projection = False
lora_mlp = False
lora_head = False

# Lessons from running 100s if not 1000s of experiments